# Associate IAM Policies with EKS Worker Nodes


In [1]:
%%bash

### Source the environment
source ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .RoleName")
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws iam list-roles \
    | jq -r ".Roles[] \
    | select(.RoleName \
    | startswith(\"eksctl-$AWS_CLUSTER_NAME\") and contains(\"NodeInstanceRole\")) \
    .Arn")
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=
export INSTANCE_PROFILE_ARN=


# Attach Roles

In [2]:
%%bash

### Source the environment
source ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

echo "Completed"

Completed


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument --role-name: expected one argument


# Associated IAM and OIDC
To use IAM Roles for Service Accounts in your cluster, you must create an OIDC identity provider in the IAM console.  

See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.

# _This may take a few minutes.  Please be patient._


In [3]:
%%bash

### Source the environment
source ~/.bash_profile

eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve


[ℹ]  eksctl version 0.19.0
[ℹ]  using region us-west-2
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 38.45833ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 72.423806ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 197.527516ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 475.03008ms
[!]  retryable error (EC2MetadataError: failed to make EC2Metadata request
	status code: 405, request id: 
caused by: ) from ec2metadata/GetToken - will retry after delay of 680.764576ms
[!]  retryable

In [4]:
%%bash

### Source the environment
source ~/.bash_profile

aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

https://oidc.eks.us-west-2.amazonaws.com/id/8C9E5AC8F46B1CE50BE16AE9BF96D7F2
Completed


# Update `~/.kube/config` with our new EKS cluster

In [5]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 

Added new context arn:aws:eks:us-west-2:853609162619:cluster/chubacluster to /home/ec2-user/.kube/config


# Verify Your EKS Cluster


In [6]:
%%bash

kubectl get ns


NAME              STATUS   AGE
default           Active   7m25s
kube-node-lease   Active   7m26s
kube-public       Active   7m26s
kube-system       Active   7m26s


In [7]:
%%bash

kubectl get nodes


NAME                                           STATUS   ROLES    AGE     VERSION
ip-192-168-11-227.us-west-2.compute.internal   Ready    <none>   2m45s   v1.15.11-eks-af3caf
ip-192-168-26-95.us-west-2.compute.internal    Ready    <none>   2m44s   v1.15.11-eks-af3caf
ip-192-168-35-205.us-west-2.compute.internal   Ready    <none>   2m50s   v1.15.11-eks-af3caf
ip-192-168-68-104.us-west-2.compute.internal   Ready    <none>   2m44s   v1.15.11-eks-af3caf
ip-192-168-77-167.us-west-2.compute.internal   Ready    <none>   2m44s   v1.15.11-eks-af3caf
